# Goals
The goal of this notebook is to get an idea of the range of parameters that result in acceptable image analysis. Ideally, the result will be a set of parameters that work for all the test images. More realistically, the result will be a range of parameters that work for the test images and some sort of criteria to distinguish which parameters work for which images.

# Approach
1. Gather statistics on quality of existing images
     - Use histograms to look at pixel values
     - Build dataframe with these descriptive statistics along with metadata
2. Choose 5 datasets that "look good" in term of descriptive statistics, eye test, and notes in metadata
3. Manually tune analysis for these datasets
4. Compare tuned values and look for correlation with descriptive statistics

### Import packagaes

In [9]:
import sys
import os
import glob

import gspread
from oauth2client.service_account import ServiceAccountCredentials

import numpy as np
import pandas as pd
from nd2reader import ND2Reader 
from scipy import ndimage as ndi
import pims
from PIL import Image

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import warnings
warnings.filterwarnings("ignore", module="matplotlib")

from ipywidgets import interact
import ipywidgets as ipw
# mpl.rcParams['figure.figsize'] = [15.0, 10.0]

### Functions for importing images and metadata

In [2]:
def loadImages(imageDir,expID):
    images = pims.open(imageDir+expID+'.nd2')
    images.bundle_axes = ['z', 'y', 'x']
#     meta = images.metadata
#     print(meta)
#     numSlices = meta['total_images_per_channel']
    
    imageArray = np.asarray(images)
    imageArray = imageArray.squeeze()
    # print(imageArray.shape)
    return imageArray

def getMetadata(imageDir,experimentID):
    ## need to think about if I want to just return all the metadata in a dataframe for later slicing and dicing
    
    
    # Access google drive spreadsheet
    scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('SenseOfTouchResearch-e5927f56c4d0.json', scope)
    c = gspread.authorize(credentials)
    metadataSpread = c.open_by_key('1LsTdPBOW79XSkk5DJv2ckiVTvpofAOOd_dJL4cgtxBQ')
    metadataFrame = pd.DataFrame(metadataSpread.sheet1.get_all_records())
    gdriveMetadata = metadataFrame.loc[metadataFrame['Experiment ID'] == 'SSN_089_001']
    
    # Access the metadata embedded in the file from the microscope
    images = pims.open(imageDir+expID+'.nd2')
    meta = images.metadata
    keysToKeep = ['height','width','date','total_images_per_channel','channels','pixel_microns']
    metadataFromScope = { keyToKeep: meta[keyToKeep] for keyToKeep in keysToKeep }
    metadataFromScope = pd.DataFrame(metadataFromScope)
    
    gdriveMetadata=gdriveMetadata.reset_index(drop=True)
    metadataFromScope=metadataFromScope.reset_index(drop=True)
    currentMetadata = metadataFromScope.join(gdriveMetadata)

    return currentMetadata

### Functions for displaying images and data

In [3]:
def dispMaxProjection(images):
    # convert data to numpy array
    maxProjection = np.amax(images,0) # collapse z axis
    maxProjection = maxProjection.squeeze()
    maxProjection = np.asarray(maxProjection)
    
    # print a bit of info for the user
    vulvaSide = thisMetadata.loc[0,'Worm vulva orientation']
    print('Worm vulva is on the '+vulvaSide+ ' side.')
    
    # figure axis setup  with increased figure size
    maxProjFig = plt.figure(figsize = (8,8)) 
    ax = maxProjFig.add_subplot(111)

    # display  image 
    ax.imshow(maxProjection, interpolation='none')
    ax.set_title('Max projection along z')
#     plt.show()
    return maxProjFig


def plotCurrentSlice(currentSlice,stack):
    # figure axis setup  with increased figure size
    imFig, ax = plt.subplots(figsize=(8,8))

    # display initial image 
    thisIm = ax.imshow(stack[currentSlice], interpolation='none')    
    return 


def scrollingFig(currentStack):
    interact(plotCurrentSlice,currentSlice = ipw.IntSlider(min=0,max=currentStack.shape[0]-1,step=1,value=0),stack = ipw.fixed(currentStack))

## 1) Get statistics on image quality

### Get list of files to analyze
The main idea here is to start building a dataframe with a bunch of info for each of the files, starting with the metadata. We start by specifying all the filenames, and then go through each file one at a time to add some new information like pixel histogram parameters and brightness levels. This section should mostly rely on the functions above to keep things clean. It could in principle be done in a big loop, but I want to actually look at the individual results to see if I can find any patterns.

In [15]:
parentDir = '/Users/adam/Documents/SenseOfTouchResearch/SSN_data/'
imageDirs = ['20181005/' , '20181008/']
expIDs = ['SSN_095_001','SSN_096_001','SSN_097_001','SSN_098_001','SSN_099_001']
    
filenames = []
for experiment in expIDs:
    thisExp = glob.glob(parentDir + '*/' + experiment + '*')
    filenames.append(thisExp)
print(filenames)

[['/Users/adam/Documents/SenseOfTouchResearch/SSN_data/20181005/SSN_095_001.nd2'], ['/Users/adam/Documents/SenseOfTouchResearch/SSN_data/20181008/SSN_096_001.nd2'], ['/Users/adam/Documents/SenseOfTouchResearch/SSN_data/20181008/SSN_097_001.nd2'], ['/Users/adam/Documents/SenseOfTouchResearch/SSN_data/20181008/SSN_098_001.nd2'], ['/Users/adam/Documents/SenseOfTouchResearch/SSN_data/20181008/SSN_099_001.nd2']]
